<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/Copy_of_finding_magic_feature_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "tapaskd123" # username from the json file
os.environ['KAGGLE_KEY'] = "aba8dc1f085221111d925003fe5a88ed" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

train_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
trn=pd.read_csv('train_transaction.csv.zip')
trn_idn=pd.read_csv('train_identity.csv.zip')

In [ ]:
trn=trn.merge(trn_idn,on='TransactionID',how='left')
import gc
del([trn_idn])
gc.collect()

In [ ]:
trn.head()

In [ ]:
trn['day']=trn['TransactionDT']//86400
for col in range(1,16):
  trn['d_'+str(col)]=trn['day']-trn['D'+str(col)]

In [ ]:
print(trn.shape)

In [ ]:
def second(data):
  return data[data.find('.'):]
def first(data):
  return data[:data.find('.')]
trn['P_emaildomain'].fillna('nan',inplace=True)
trn['R_emaildomain'].fillna('nan',inplace=True)
trn['P_emaildomain_first']=trn['P_emaildomain'].apply(first)
trn['P_emaildomain_second']=trn['P_emaildomain'].apply(second)
trn['R_emaildomain_first']=trn['R_emaildomain'].apply(first)
trn['R_emaildomain_second']=trn['R_emaildomain'].apply(second)

In [ ]:
trn=trn.drop(['TransactionID','TransactionDT','isFraud'],1)
cat=list(trn.select_dtypes(include=object))
trn[cat]=trn[cat].fillna('nan')
trn=trn.fillna(-999)

In [ ]:
trn=trn.loc[trn['D1']<trn['day']].reset_index(drop=True)

In [ ]:
from tqdm import tqdm
import random
def group(df,cols):
  df['id']=''
  for col in cols:
    df['id']+=df[col].astype(str)
  result=0
  for _,sdo in df.groupby(['id']):
    tmp=0
    tmp+=sdo['TransactionAmt'].values.sum()
    tmp-=sdo.tail(1)['V307'].values
    if tmp>0:
      result+=tmp
    else:
      tmp-=result
  return result
trn=trn
cols=list(trn)
cols.remove('TransactionAmt')
cols.remove('V307')
random.shuffle(cols)
ans=0
ls=[]
dk={}
for en,col in enumerate(tqdm(cols)):
  ls.append(col)
  n_ans=group(trn,ls)
  dk[col]=ans-n_ans
  if ans!=0:
    if n_ans>=ans:
      ls.remove(col)
    else:
      ans=n_ans
  else:
    ans=n_ans

In [ ]:
ans

In [ ]:
df=pd.DataFrame(dk).T
df['rank']=list(range(450))
df.sort_values(by=0,ascending=False).head(20)

In [ ]:
ls